### Load the data to Landing zone and move it to Bronze layer

In [0]:

select current_catalog()

In [0]:
%sql
select current_database();

In [0]:

create volume if not exists dlt_volume;

In [0]:
%python
dbutils.fs.mkdirs('/Volumes/workspace/default/dlt_volume/landing')

In [0]:
%python
dbutils.fs.mkdirs('/Volumes/workspace/default/dlt_volume/landing/orders')

In [0]:
%python
dbutils.fs.mkdirs('/Volumes/workspace/default/dlt_volume/landing/customers')

In [0]:
%python
display(dbutils.fs.ls('/Volumes/workspace/default/dlt_volume/landing'))

### in case of Azure , create a adls account and container landing and directoreis orders & customers
#### in databricks I have craeted a valume and upload data 

In [0]:

create streaming table if not exists orders_bronze
as
select *, current_timestamp() as load_time,
_metadata.file_path as file_name
from stream(cloud_files('/Volumes/workspace/default/dlt_volume/landing/orders', 'csv', map('cloudFiles.inferColumnTypes', 'true')));

create streaming table if not exists customers_bronze
as
select *, current_timestamp() as load_time,
input_file_name() as file_name
from stream(cloud_files('/Volumes/workspace/default/dlt_volume/landing/customers', 'csv', map('cloud_files.inferColumnTypes', 'true')));




### Step 2 : Take the data from Bronze and clean it and load it in the silver_cleaned

In [0]:
create streaming table if not exists orders_cleaned_silver(
constraint valid_orders expect(order_id is not null) on violation drop row,
constraint valid_customers expect(customer_id is not null) on violation drop row
)

as
select OrderID as order_id,
OrderDate as order_date,
CustomerID as customer_id,
TotalAmount as total_amount,
Status as order_status,
file_name,
load_time
from 
stream(live.orders_bronze);

create streaming table if not exists customers_cleaned_silver(
constraint valid_orders expect(order_id is not null) on violation drop row,
constraint valid_customers expect(customer_id is not null) on violation drop row
)
as
select CustomerID as customer_id,
CustomerName as customer_name,
ContactNumber as contact_number,
Email as email,
Address as address,
DateOfBirth as date_of_birth,
RegistrationDate as resistration_date,
file_name,
load_time
from 
stream(live.customers_bronze)



### take the data from cleaned silver and handle duplicates , scd 2 and load it to silver 

In [0]:
create streaming table orders_silver;

apply changes into orders_silver
from stream(live.orders_cleaned_silver)
keys(order_id)
sequence by load_time;

create streaming table customer_silver;

apply changes into customer_silver
from stream(live.customers_cleaned_silver)
keys(customer_id)
sequence by load_time
stored as scd type 2;

### Gold Layer : Take the date from silver layer and perform aggregations and create Gold layer

In [0]:
create materialized view if not exists city_wise_sales_gold
as
select city , sum(total_amount) as total_sales
from orders_silver o
join customer_silver c
on o.customer_id = c.customer_id
group by city;